# Case Study - News Categorization

학습 목표

이번 강의에서는 지금까지 배운 List Comprehension, Data Structure(Collections), Pythonic Code 작성법 등을 기반으로 다른 패키지 사용 없이 Python 만을 이용하여 한 뉴스에 대해 비슷한 뉴스를 찾아내도록 작성된 코드를 실행하면서, 설명하는 방식으로 진행되는 실습 위주 강의입니다.

핵심 키워드

- Categorization
- One-hot Encoding
- Bag of words
- Cosine Distance
- Corpus
- List Comprehension

컴퓨터는 문자를 그래도 이해하지 못한다.

따라서 문자를 숫자로 바꿔줘야 한다.

그러면 숫자로 유사하다는 것은 어떻게 표현할까 ?

유사하다 => 가깝다

```문자 -> 숫자 -> Vector```

## 문자를 Vector로 – One-hot Encoding
- 하나의 단어를 Vector의 Index로 인식, 단어 존재시 1 없으면 0

## Bag of words
- 단어별로 인덱스를 부여해서, 한 문장(또는 문서)의 단어의 개수를 Vector로 표현

# 유사성
Distance measure
- 고등학교때 배운 2차원 평면상 거리측정 방법들
    - ex) Cosine Similarity, Euclidean Distance, Manhattan distance

## Euclidian distance
- 피타고라스 정리, 두 점 사이의 직선의 거리

## Cosine distance
- 두 점 사이의 각도
- Why cosine similarity? Count < Direction
- Love,hate (5,0), (5,4), (4,0) , 어느점이 가장 가까운가

# News Categorization 실습
- 축구와 야구 선수들의 영문 기사를 분류해보자

## Process
- 파일을 불러오기
- 파일을 읽어서 단어사전 (corpus) 만들기
- 단어별로 Index 만들기
- 만들어진 인덱스로 문서별로 Bag of words vector 생성
- 비교하고자 하는 문서 비교하기
- 얼마나 맞는지 측정하기

### 파일 불러오기

In [2]:
import os

def get_file_list(dir_name) :
    return os.listdir(dir_name)

if __name__ == '__main__' :
    dir_name = "data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]

In [6]:
print(file_list[0])
print(len(file_list))

data\1_Dae-Ho Lee walk-off homer gives Mariners 4-2 win over Rangers.txt
80


## 파일별로 내용 읽기

In [7]:
def get_conetents(file_list):
    y_class = []
    X_text = []
    class_dict = {1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"}

    for file_name in file_list:
        try:
            f = open(file_name, "r", encoding="cp949")
            category = int(file_name.split(os.sep)[1].split("_")[0])
            y_class.append(class_dict[category])
            X_text.append(f.read())
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class

## Corpus 만들기 + 단어별 index 생성하기

In [9]:
def get_cleaned_text(word):
    import re
    word = re.sub('\W+', '', word.lower() )
    return word

def get_corpus_dict(text):
    text = [sentence.split() for sentence in text]
    clenad_words = [get_cleaned_text(word) for words in text for word in words]
    
    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i, v in enumerate(set(clenad_words)):
        corpus_dict[v] = i
    return corpus_dict

## 문서별로 Bag of words vector 생성

In [10]:
def get_count_vector(text, corpus):
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words] for words in text]
    X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))]
    
    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
    return X_vector

## 비교하기

In [11]:
import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

## 비교결과 정리하기

In [12]:
def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(get_cosine_similarity(source_vector, target_vector))
    return similarity_list

def get_top_n_similarity_news(similarity_score, n):
    import operator
    x = {i:v for i, v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))
    return list(reversed(sorted_x))[1:n+1]

## 성능 측정하기

In [15]:
def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]
    return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)

if __name__ == "__main__":
    dir_name = "data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]

    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)
    source_number = 10

    result = []

    for i in range(80):
        source_number = i

        similarity_score = get_similarity_score(X_vector, source_number)
        similarity_news = get_top_n_similarity_news(similarity_score, 10)
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(sum(result) / 80)

Number of words : 4024
0.6950000000000001
